In [5]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)


Title: Predicting the Chances of a Stroke

Introduction: 

Strokes are serious life-threatening medical conditions that happens when the blood supply to part of the brain is cut off. It is imperative that strokes are met with immediate treatment to prevent brain damage. According to the World Health Organization (WHO), strokes are the 2nd leading cause of death globally, responsible for approximately 11% of total deaths. 

Our project intends to accurately predict the chances of a stroke occuring in individuals by examining a variety of input parameters. (need to narrow this down)

Stroke Prediction Dataset, the title of the dataset we chose for this project, was adapted from the kaggle data scientist fedesoriano. The dataset provides ten clinical features for predicting strokes: gender, age, hypertension, heart_disease, ever_married, work_type, residence_type, avg_glucose_level, bmi, and smoking_status.

link to dataset: (https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset?resource=download)

Preliminary exploratory data analysis:

In [6]:
set.seed(1234) #randomizes rows

stroke_data <- read_csv("data/stroke_data.csv") #reading dataset in


stroke_split <- initial_split(stroke_data, prop = 0.75, strata = stroke) #split data into training and testing sets
    stroke_train <- training(stroke_split)
    stroke_test <- testing(stroke_split)

stroke_train
stroke_test

Rows: 5110 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): gender, ever_married, work_type, Residence_type, bmi, smoking_status
dbl (6): id, age, hypertension, heart_disease, avg_glucose_level, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
19723,Female,35,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
37544,Male,51,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
44679,Female,44,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
56669,Male,81,0,0,Yes,Private,Urban,186.21,29,formerly smoked,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
36901,Female,45,0,0,Yes,Private,Urban,97.95,24.5,Unknown,0
14180,Female,13,0,0,No,children,Rural,103.08,18.6,Unknown,0
18234,Female,80,1,0,Yes,Private,Urban,83.75,N/A,never smoked,0


Methods:

Expected outcomes and significance:

There are simply too many variables to start out with to make a completely accurate prediction. But if we were to expect outcomes, individuals that have a heart disease, smoke regularly, and maintain high average glucose levels are at most risk of a stroke.

These findings could give more guidance as how to accurately predict strokes in unsuspecting individuals. In other words, what factors should we be paying more attention to in regards to the increase in possibility of an individual suffering a stroke. 

This project can lead to future questions about what is the most important factor in stress volume.